In [ ]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader
import os
from text_writer import *
import pickle
from w2v_functions import *
import selection
from matplotlib import pyplot as plt

In [ ]:
#load Google model
f = open('mymodel.pkl', 'rb')
google_model = pickle.load(f)
f.close()

In [ ]:
#load Wiki model
wiki_model = Word2Vec.load('wiki_model.model')

In [ ]:
#create new lists of words for the analysis

list_E = ['environment', 'sustainability', 'climate', 'renewable', 'emission', 'pollution', 'recycle']

list_S = ['social', 'disabled', 'health', 'discrimination', 'equality', 'community', 'diversity']

list_G = ['governance', 'audit', 'leadership', 'compensation', 'transparency', 'vote', 'stakeholder']

#add lists to a dictionary
ESG_dict = {'Environmental': list_E, 'Social': list_S, 'Governance': list_G}
topic_names = ['Environmental', 'Social', 'Governance']

In [ ]:
#Import data as .txt

#Choose type of data
types = ['10-K', 'DEF 14A']
#Choose Companies
pa = os.getcwd().replace('\\', '/') +'/Top 40'
print(os.path.exists(pa))
tickers, years_for_each = selection.find_good(pa)
tickers.append('WFC')
tickers.append('BLK')
#print(years_for_each)
print(tickers)

#Choose number of years
nb = 10

Tech = ['AAPL','MSFT','NVDA','ADBE','INTC','ORCL','QCOM']
Telecom = ['VZ','CSCO','CMCSA','NFLX','TMUS']
Banking = ['JPM','BAC','BLK','WFC']
Retail = ['AMZN','WMT','NKE','KO']
Health = ['UNH','ABT','MRK','PFE','TMO']
sect = [Tech, Telecom, Banking, Retail, Health]

#import data
#import_data(nb, tickers, types, First=False)

In [ ]:
#select stopwords
nonotext = open(os.getcwd().replace("\\","/") +"/nonosquare.txt","r",encoding='utf-8')
nonolist = nonotext.readlines()[0].split(" ")

#select sub-set of data
tickerselect = ['NFLX']
types = ['10-K', 'DEF 14A']
print(tickerselect)

In [ ]:
#Compute scores using w2v
models = [wiki_model.wv]
model_names = ['Wiki Model']

for model, model_name in zip(models, model_names):
    #Compute scores for each topic (each company, each year)
    for topic, name in zip(ESG_dict.keys(), topic_names):
        topic_list = ESG_dict[topic]
        
        for tick in tickerselect:
            
            #compute score for 10K
            score_10K = score_w2v(model, topic_list, tick, "10-K", nonolist)
            #compute score for Proxy Stat.
            score_proxy = score_w2v(model, topic_list, tick, "DEF 14A", nonolist)
            #compute weighted mean between scores
            merger = merge_w2v(score_10K,score_proxy,wP=0.5,wK=0.5)
            #plot scores
            X,Y = plotting(merger)
            plt.plot(X,Y,label = tick)
            
        plt.legend()
        plt.title(name+' '+'('+model_name+')')  
        plt.show()

In [ ]:
score_path = os.getcwd().replace('\\', '/') + '/score_files_w2v'

In [ ]:
model = wiki_model.wv

for tick in tickers:
    if os.path.exists(score_path + '/' + tick + '.txt'):
        continue
    else:
        print(tick)
        file_name = score_path + '/' + tick + '.txt'
        score_tick = pd.DataFrame()
        index = None
        for topic, topic_name in zip(ESG_dict.keys(), topic_names):
            topic_list = ESG_dict[topic]
            score_10K = score_w2v(model, topic_list, tick, "10-K", nonolist)
            score_proxy = score_w2v(model, topic_list, tick, "DEF 14A", nonolist)
            merger = merge_w2v(score_10K,score_proxy,wP=0.5,wK=0.5)
            score_tick[topic_name] = [val for val in merger.values()]
            index = [key[-2:] for key in merger.keys()]

        score_tick.index.name = 'Year'
        score_tick.index = index   
        new_text_file = open(file_name, "w", encoding = 'utf-8') 
        new_text_file.write(score_tick.to_string()) 
        new_text_file.close()